In [ ]:
from pytrends.request import TrendReq
import pandas as pd
import sqlite3
import numpy as np

In [ ]:
#obtendo as tabelas de indicadores
conn =sqlite3.connect("indicadores.db")
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())
brasilDf=pd.read_sql_query("select * from brazil_df",conn)
conn.close()

In [ ]:
#lendo o arquivo com as keywords
keywords=pd.read_csv('Queries categorizadas.csv')
keywords

In [ ]:
#gerando a lista de keywords
kw_list=[]
for i in range(len(keywords)):
    keywords['query'].iloc[i]=keywords['query'].iloc[i].replace('[','')
    keywords['query'].iloc[i]=keywords['query'].iloc[i].replace(']','')
    keywords['query'].iloc[i]=keywords['query'].iloc[i].replace('\'','')
    kw_list.append(keywords['query'].iloc[i])

In [ ]:
#como pode ser visto,a ultima semana totalmente em 2020 termina no dia 26 de dezembro e a primeira com dados
#começa dia 23 de fevereiro
print(brasilDf['week_last_day'].head())
print(brasilDf['week_last_day'].tail())

In [ ]:
pytrends = TrendReq(hl='en-US', tz=360)

In [ ]:
#dados das semanas 2020-02-23 até 2020-12-26 que são as semanas epidemiologicas 9 até a 52 relativos ao brasil
df=pd.DataFrame()
for i in range(len(kw_list)):
    kw = []
    kw.append(kw_list[i])
    pytrends.build_payload(kw, cat=0, timeframe='2020-02-23 2020-12-26', geo='BR', gprop='')
    aux=pytrends.interest_over_time()
    df[kw_list[i]]=aux[kw_list[i]]

In [ ]:
df.to_csv('SeriesKeywords.csv')

In [ ]:
#dados das semanas 2020-02-23 até 2020-12-26 que são as semanas epidemiologicas 9 até a 52 relativos a MG
dfMg=pd.DataFrame()
for i in range(len(kw_list)):
    kw = []
    kw.append(kw_list[i])
    pytrends.build_payload(kw, cat=0, timeframe='2020-02-23 2020-12-26', geo='BR-MG', gprop='')
    aux=pytrends.interest_over_time()
    #caso a consulta não consiga resultado, criará uma coluna de NAN
    if not aux.empty:
        dfMg[kw_list[i]]=aux[kw_list[i]]
    else:
        dfMg[kw_list[i]]=np.nan

In [ ]:
dfMg.to_csv('SeriesKeywordsMg.csv')